# 01 - Data Loading & Cleaning

**Objective**: Load and clean the raw flight delay dataset (2018-2024) to prepare for analysis.

**Steps**:
1. Load raw data
2. Remove columns with excessive missing values
3. Remove target leakage columns
4. Create binary target variable (DELAYED: 1 if DepDelay > 0, else 0)
5. Remove post-departure information
6. Save cleaned dataset

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 120)
sns.set()


In [2]:
# Load raw dataset
data_path = 'data/flight_data_2018_2024_raw.csv'
df = pd.read_csv(data_path, low_memory=False)
df.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Marketing_Airline_Network,Operated_or_Branded_Code_Share_Partners,DOT_ID_Marketing_Airline,IATA_Code_Marketing_Airline,Flight_Number_Marketing_Airline,Originally_Scheduled_Code_Share_Airline,DOT_ID_Originally_Scheduled_Code_Share_Airline,IATA_Code_Originally_Scheduled_Code_Share_Airline,Flight_Num_Originally_Scheduled_Code_Share_Airline,Operating_Airline,DOT_ID_Operating_Airline,IATA_Code_Operating_Airline,Tail_Number,Flight_Number_Operating_Airline,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCityName,DestState,DestStateFips,DestStateName,DestWac,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,Cancelled,CancellationCode,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,FirstDepTime,TotalAddGTime,LongestAddGTime,DivAirportLandings,DivReachedDest,DivActualElapsedTime,DivArrDelay,DivDistance,Div1Airport,Div1AirportID,Div1AirportSeqID,Div1WheelsOn,Div1TotalGTime,Div1LongestGTime,Div1WheelsOff,Div1TailNum,Div2Airport,Div2AirportID,Div2AirportSeqID,Div2WheelsOn,Div2TotalGTime,Div2LongestGTime,Div2WheelsOff,Div2TailNum,Div3Airport,Div3AirportID,Div3AirportSeqID,Div3WheelsOn,Div3TotalGTime,Div3LongestGTime,Div3WheelsOff,Div3TailNum,Div4Airport,Div4AirportID,Div4AirportSeqID,Div4WheelsOn,Div4TotalGTime,Div4LongestGTime,Div4WheelsOff,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Duplicate,Unnamed: 119
0,2024,1,1,14,7,2024-01-14,UA,UA_CODESHARE,19977,UA,4432,NaN,NaN,NaN,NaN,G7,20500,G7,N535GJ,4432,13296,1329605,30721,MHT,"Manchester, NH",NH,33,New Hampshire,14,11618,1161802,31703,EWR,"Newark, NJ",NJ,34,New Jersey,21,1738,1849.0,71.0,71.0,1.0,4.0,1700-1759,24.0,1913.0,2001.0,6.0,1922,2007.0,45.0,45.0,1.0,3.0,1900-1959,0.0,NaN,0.0,104.0,78.0,48.0,1.0,209.0,1,0.0,0.0,0.0,0.0,45.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
1,2024,1,1,14,7,2024-01-14,UA,UA_CODESHARE,19977,UA,4430,NaN,NaN,NaN,NaN,G7,20500,G7,N535GJ,4430,12264,1226402,30852,IAD,"Washington, DC",VA,51,Virginia,38,11618,1161802,31703,EWR,"Newark, NJ",NJ,34,New Jersey,21,815,814.0,-1.0,0.0,0.0,-1.0,0800-0859,58.0,912.0,953.0,7.0,934,1000.0,26.0,26.0,1.0,1.0,0900-0959,0.0,NaN,0.0,79.0,106.0,41.0,1.0,212.0,1,0.0,0.0,26.0,0.0,0.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
2,2024,1,1,14,7,2024-01-14,UA,UA_CODESHARE,19977,UA,4429,NaN,NaN,NaN,NaN,G7,20500,G7,N535GJ,4429,11618,1161802,31703,EWR,"Newark, NJ",NJ,34,New Jersey,21,13296,1329605,30721,MHT,"Manchester, NH",NH,33,New Hampshire,14,1540,1654.0,74.0,74.0,1.0,4.0,1500-1559,41.0,1735.0,1816.0,6.0,1656,1822.0,86.0,86.0,1.0,5.0,1600-1659,0.0,NaN,0.0,76.0,88.0,41.0,1.0,209.0,1,0.0,74.0,12.0,0.0,0.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
3,2024,1,1,14,7,2024-01-14,UA,UA_CODESHARE,19977,UA,4428,NaN,NaN,NaN,NaN,G7,20500,G7,N547GJ,4428,15016,1501606,31123,STL,"St. Louis, MO",MO,29,Missouri,64,13930,1393008,30977,ORD,"Chicago, IL",IL,17,Illinois,41,630,630.0,0.0,0.0,0.0,0.0,0600-0659,47.0,717.0,808.0,15.0,758,823.0,25.0,25.0,1.0,1.0,0700-0759,0.0,NaN,0.0,88.0,113.0,51.0,1.0,258.0,2,0.0,0.0,25.0,0.0,0.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [3]:
df.dtypes

Year                  int64
Quarter               int64
Month                 int64
DayofMonth            int64
DayOfWeek             int64
                     ...   
Div5LongestGTime    float64
Div5WheelsOff       float64
Div5TailNum         float64
Duplicate            object
Unnamed: 119        float64
Length: 120, dtype: object

In [4]:
# Basic info
df.shape, df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 582425 entries, 0 to 582424
Columns: 120 entries, Year to Unnamed: 119
dtypes: float64(70), int64(23), object(27)
memory usage: 533.2+ MB


((582425, 120), None)

In [5]:
# Check missing values
df.isna().sum().sort_values(ascending=False).head(20)

Unnamed: 119        582425
Div5Airport         582425
Div4WheelsOff       582425
Div4WheelsOn        582425
Div4AirportSeqID    582425
Div4AirportID       582425
Div4Airport         582425
Div4TailNum         582425
Div5AirportID       582425
Div4TotalGTime      582425
Div5AirportSeqID    582425
Div5WheelsOn        582425
Div5TotalGTime      582425
Div5LongestGTime    582425
Div5WheelsOff       582425
Div5TailNum         582425
Div4LongestGTime    582425
Div3AirportID       582424
Div3TotalGTime      582424
Div3WheelsOn        582424
dtype: int64

In [6]:
df.columns

Index(['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'FlightDate', 'Marketing_Airline_Network',
       'Operated_or_Branded_Code_Share_Partners', 'DOT_ID_Marketing_Airline', 'IATA_Code_Marketing_Airline',
       ...
       'Div5Airport', 'Div5AirportID', 'Div5AirportSeqID', 'Div5WheelsOn', 'Div5TotalGTime', 'Div5LongestGTime',
       'Div5WheelsOff', 'Div5TailNum', 'Duplicate', 'Unnamed: 119'],
      dtype='object', length=120)

In [7]:
# Drop columns with more than 90% missing values
missing_ratio = df.isna().mean()
cols_to_drop = missing_ratio[missing_ratio > 0.90].index

df = df.drop(columns=cols_to_drop)

df.shape


(582425, 67)

In [8]:
df.columns

Index(['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'FlightDate', 'Marketing_Airline_Network',
       'Operated_or_Branded_Code_Share_Partners', 'DOT_ID_Marketing_Airline', 'IATA_Code_Marketing_Airline',
       'Flight_Number_Marketing_Airline', 'Operating_Airline ', 'DOT_ID_Operating_Airline',
       'IATA_Code_Operating_Airline', 'Tail_Number', 'Flight_Number_Operating_Airline', 'OriginAirportID',
       'OriginAirportSeqID', 'OriginCityMarketID', 'Origin', 'OriginCityName', 'OriginState', 'OriginStateFips',
       'OriginStateName', 'OriginWac', 'DestAirportID', 'DestAirportSeqID', 'DestCityMarketID', 'Dest', 'DestCityName',
       'DestState', 'DestStateFips', 'DestStateName', 'DestWac', 'CRSDepTime', 'DepTime', 'DepDelay',
       'DepDelayMinutes', 'DepDel15', 'DepartureDelayGroups', 'DepTimeBlk', 'TaxiOut', 'WheelsOff', 'WheelsOn',
       'TaxiIn', 'CRSArrTime', 'ArrTime', 'ArrDelay', 'ArrDelayMinutes', 'ArrDel15', 'ArrivalDelayGroups',
       'ArrTimeBlk', 'Cancelled',

In [9]:
df.isna().sum().sort_values(ascending=False).head(20)

LateAircraftDelay       447850
SecurityDelay           447850
NASDelay                447850
WeatherDelay            447850
CarrierDelay            447850
AirTime                  23710
ArrDelay                 23710
ArrDelayMinutes          23710
ArrDel15                 23710
ArrivalDelayGroups       23710
ActualElapsedTime        23710
WheelsOn                 22332
TaxiIn                   22332
ArrTime                  22331
TaxiOut                  21910
WheelsOff                21910
DepartureDelayGroups     21470
DepDel15                 21470
DepDelayMinutes          21470
DepDelay                 21470
dtype: int64

In [10]:
delay_reason_cols = [
    'LateAircraftDelay', 'SecurityDelay', 'NASDelay',
    'WeatherDelay', 'CarrierDelay'
]

df = df.drop(columns=delay_reason_cols)


In [11]:
arrival_cols = [
    'ArrDelay', 'ArrDelayMinutes', 'ArrDel15',
    'ArrivalDelayGroups', 'ActualElapsedTime',
    'AirTime', 'WheelsOn', 'TaxiIn', 'ArrTime'
]

df = df.drop(columns=arrival_cols)


In [12]:
df.isna().sum().sort_values(ascending=False).head(15)


WheelsOff               21910
TaxiOut                 21910
DepartureDelayGroups    21470
DepDelay                21470
DepDelayMinutes         21470
DepDel15                21470
DepTime                 21396
Tail_Number              6276
Year                        0
DestCityName                0
DestState                   0
DestStateFips               0
DestStateName               0
DestWac                     0
CRSDepTime                  0
dtype: int64

In [13]:
# Create binary target: 1 = Delayed, 0 = On-time
df['DELAYED'] = (df['DepDelay'] > 0).astype(int)

df['DELAYED'].value_counts(normalize=True)


DELAYED
0    0.616778
1    0.383222
Name: proportion, dtype: float64

In [14]:
target_leakage_cols = [
    'DepDelay', 'DepDelayMinutes',
    'DepDel15', 'DepartureDelayGroups'
]

df = df.drop(columns=target_leakage_cols)


In [15]:
post_departure_cols = ['WheelsOff', 'TaxiOut', 'DepTime']
df = df.drop(columns=post_departure_cols)


In [16]:
df = df.drop(columns=['Tail_Number'])


In [17]:
df.isna().sum().sort_values(ascending=False).head(10)


Year                0
DepTimeBlk          0
DestAirportSeqID    0
DestCityMarketID    0
Dest                0
DestCityName        0
DestState           0
DestStateFips       0
DestStateName       0
DestWac             0
dtype: int64

In [18]:
clean_path = "data/flight_data_2018_2024_cleaned.csv"
df.to_csv(clean_path, index=False)

clean_path


'data/flight_data_2018_2024_cleaned.csv'